<a href="https://colab.research.google.com/github/pipaber/GNN/blob/main/proteinbert2GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nadavbra/protein_bert


Cloning into 'protein_bert'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 249 (delta 83), reused 61 (delta 61), pack-reused 152
Receiving objects: 100% (249/249), 23.44 MiB | 21.01 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [ ]:
%cd protein_bert
%ls
!git submodule init
!git submodule update
!python setup.py install

/content/protein_bert
 bin/   protein_benchmarks/   proteinbert/  'ProteinBERT demo.ipynb'   README.md   setup.py
Submodule 'proteinbert/shared_utils' (https://github.com/nadavbra/shared_utils.git) registered for path 'proteinbert/shared_utils'
Cloning into '/content/protein_bert/proteinbert/shared_utils'...
Submodule path 'proteinbert/shared_utils': checked out 'dc1c62a1754c51f6d46b7486e4a3e5e62c0570e1'
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.init

In [ ]:
from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
pretrained_model_generator, input_encoder = load_pretrained_model()

 Local model dump file /root/proteinbert_models/default.pkl doesn't exist. Will download https://media.githubusercontent.com/media/nadavbra/proteinbert_data_files/master/epoch_92400_sample_23500000.pkl into /root/proteinbert_models. Please approve or reject this (to exit and potentially call the function again with different parameters).
Do you approve downloading the file into the specified directory? Please specify "Yes" or "No":Yes
Downloaded file: /root/proteinbert_models/epoch_92400_sample_23500000.pkl
Created: /root/proteinbert_models/default.pkl


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
def get_embeddings(seq, seq_len=512, batch_size=1):

    model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(seq_len=seq_len))
    encoded_x = input_encoder.encode_X(seq, seq_len)
    local_representations, global_representations = model.predict(encoded_x, batch_size=batch_size)

    return local_representations, global_representations

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO, Entrez

protein_ids = ["P0DTC2", "Q9Y263", "P0DTD1", "Q8NBP7", "Q96QV1", "XP_027089409.1"]

def fetch_and_save_proteins(protein_ids, output_file):
    with Entrez.efetch(db="protein", id=",".join(protein_ids), rettype="fasta", retmode="text") as handle:
        with open(output_file, "w") as fasta_file:
            fasta_file.write(handle.read())

def read_proteins_from_fasta(fasta_file):
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
    return sequences

def get_max_sequence_length(fasta_file):
    max_length = 0
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_length = len(record.seq)
        if seq_length > max_length:
            max_length = seq_length
    return max_length

# File to save the fetched protein sequences
output_file = "proteins.fasta"


fetch_and_save_proteins(protein_ids, output_file)

# Read the protein sequences from the FASTA file
protein_sequences = read_proteins_from_fasta(output_file)

# Print the first few sequences to verify
for seq in protein_sequences[:5]:
    print(seq)

print(len(max(protein_sequences)))

/usr/local/lib/python3.10/dist-packages/Bio/Entrez/__init__.py:723: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGR

In [ ]:
batch_size = 2  # Adjust based on your GPU memory
seq_len = get_max_sequence_length(output_file)+2
global_embeds = []
local_embeds = []

for i in range(0, len(protein_sequences), batch_size):
    batch_seqs = protein_sequences[i:i + batch_size]
    local_representation, global_representation = get_embeddings(batch_seqs, seq_len=seq_len, batch_size=batch_size)
    global_embeds.extend(global_representation)
    local_embeds.extend(local_representation)

# Convert to numpy array if needed
import numpy as np
global_embeds = np.array(global_embeds)
local_embeds = np.array(local_embeds)

print(f'Global embeddings shape: {global_embeds.shape}')
print(f'Local embeddings shape: {local_embeds.shape}')

1/1 [==============================] - 2s 2s/step
Global embeddings shape: (6, 15599)
Local embeddings shape: (6, 7098, 1562)
